# UN World Population Prospects 2022
[_Source data_](https://population.un.org/wpp/)

## Parameters

In [1]:
dest_dir = "/tmp/un_wpp"

## Imports

In [2]:
import zipfile
import tempfile
import shutil
from pathlib import Path
import os

import requests
import pandas as pd

from owid.catalog import Dataset, Table, frames, LocalCatalog
from owid.walden import Catalog
from etl.paths import DATA_DIR
from etl.steps.data.converters import convert_walden_metadata

## Fetch walden dataset

In [3]:
walden_ds = Catalog().find_one("un", "2022", "un_wpp")

In [482]:
type(ds)

owid.catalog.datasets.Dataset

## Make a dataset

In [445]:
ds = Dataset.create_empty(dest_dir)
ds.metadata = convert_walden_metadata(walden_ds)
ds.metadata.short_name = "un_wpp"
ds.save()

## Unzip to temp directory

In [6]:
tmp_dir = tempfile.mkdtemp()
z = zipfile.ZipFile(walden_ds.local_path)
z.extractall(tmp_dir)

In [429]:
# os.listdir(tmp_dir)

## Load datasets

In [101]:
from pandas.api.types import CategoricalDtype

### Population

In [408]:
filenames = list(filter(lambda x: "PopulationBySingleAgeSex" in x, os.listdir(tmp_dir)))
dtype = {
    "SortOrder": "category",
    "LocID": "category",
    "Notes": "category",
    "ISO3_code": "category",
    "ISO2_code": "category",
    "SDMX_code": "category",
    "LocTypeID": "category",
    "LocTypeName": "category",
    "ParentID": "category",
    "Location": "category",
    "VarID": CategoricalDtype(categories=["2", "3", "4"]),
    "Variant": CategoricalDtype(categories=["Medium", "High", "Low"]),
    "Time": "int",
    "MidPeriod": "int",
    "AgeGrp": "category",
    "AgeGrpStart": "int",
    "AgeGrpSpan": "int",
    "PopMale": "float",
    "PopFemale": "float",
    "PopTotal": "float",
}
df_population = pd.concat(
    [
        pd.read_csv(os.path.join(tmp_dir, filename), dtype=dtype)
        for filename in filenames
    ],
    ignore_index=True,
)

### Fertility

In [410]:
(filename,) = [
    f
    for f in filter(lambda x: "Fertility" in x, os.listdir(tmp_dir))
    if "notes" not in f
]
dtype = {
    "SortOrder": "category",
    "LocID": "category",
    "Notes": "category",
    "ISO3_code": "category",
    "ISO2_code": "category",
    "SDMX_code": "category",
    "LocTypeID": "category",
    "LocTypeName": "category",
    "ParentID": "category",
    "Location": "category",
    "VarID": "category",
    "Variant": "category",
    "Time": "int",
    "MidPeriod": "float",
    "AgeGrp": "category",
    "AgeGrpStart": "int",
    "AgeGrpSpan": "int",
    "ASFR": "float",
    "PASFR": "float",
    "Births": "float",
}
df_fertility = pd.read_csv(os.path.join(tmp_dir, filename), dtype=dtype)

## Demographic

In [440]:
filenames = [
    f
    for f in filter(lambda x: "Demographic" in x, os.listdir(tmp_dir))
    if "notes" not in f
]
dtype = {
    "SortOrder": "category",
    "LocID": "category",
    "Notes": "category",
    "ISO3_code": "category",
    "ISO2_code": "category",
    "SDMX_code": "category",
    "LocTypeID": "category",
    "LocTypeName": "category",
    "ParentID": "category",
    "Location": "category",
    "VarID": CategoricalDtype(
        categories=["2", "3", "4", "5", "6", "7", "8", "9", "10", "16"]
    ),
    "Variant": CategoricalDtype(
        categories=[
            "Medium",
            "High",
            "Low",
            "Constant fertility",
            "Instant replacement",
            "Zero migration",
            "Constant mortality",
            "No change",
            "Momentum",
        ]
    ),
    "Time": "int",
}
df_demographics = pd.concat(
    [
        pd.read_csv(os.path.join(tmp_dir, filename), dtype=dtype)
        for filename in filenames
    ],
    ignore_index=True,
)

### Excel: deaths & dependency ratio

In [279]:
dtype_base = {
    "Index": "category",
    "Variant": "category",
    "Region, subregion, country or area *": "category",
    "Notes": "category",
    "Location code": "category",
    "ISO3 Alpha-code": "category",
    "ISO2 Alpha-code": "category",
    "SDMX code**": "category",
    "Type": "category",
    "Parent code": "category",
    "Year": "int",
}
kwargs = {
    "skiprows": 16,
    "sheet_name": ["Estimates", "Medium variant", "High variant", "Low variant"],
}


def read_file(filename):
    # print(filename)
    # Load excel
    df = pd.read_excel(os.path.join(tmp_dir, filename), **kwargs)
    # Check
    assert len(set(str(set(d.columns)) for _, d in x.items())) == 1
    # Concatenate
    df = pd.concat(list(df.values()), ignore_index=True)
    # Filter
    df = df[df.Type != "Label/Separator"]
    # Dtypes
    dtype = {**dtype_base, **{m: "float" for m in df.columns if m not in dtype_base}}
    df = df.astype(dtype)
    if "BOTH_SEXES" in filename:
        df = df.assign(Sex="Both")
    elif "FEMALE" in filename:
        df = df.assign(Sex="Female")
    else:
        df = df.assign(Sex="Male")
    return df

In [281]:
filenames = list(filter(lambda x: "DEATHS" in x, os.listdir(tmp_dir)))
# Load
dfs = [read_file(filename) for filename in filenames]
df_deaths = pd.concat(dfs, ignore_index=True)

WPP2022_MORT_F01_1_DEATHS_SINGLE_AGE_BOTH_SEXES.xlsx
WPP2022_MORT_F01_2_DEATHS_SINGLE_AGE_MALE.xlsx
WPP2022_MORT_F01_3_DEATHS_SINGLE_AGE_FEMALE.xlsx


In [282]:
filenames = list(filter(lambda x: "DEPENDENCY_RATIOS" in x, os.listdir(tmp_dir)))
# Load
dfs = [read_file(filename) for filename in filenames]
df_depratio = pd.concat(dfs, ignore_index=True)

WPP2022_POP_F07_3_DEPENDENCY_RATIOS_FEMALE.xlsx
WPP2022_POP_F07_2_DEPENDENCY_RATIOS_MALE.xlsx
WPP2022_POP_F07_1_DEPENDENCY_RATIOS_BOTH_SEXES.xlsx


## Clean dataset

### Sanity checks

In [320]:
def post_processing(
    df,
    column_location,
    column_location_id,
    column_location_type=None,
    location_type_values=None,
):
    # There are some duplicates. Some locations appear with two different location IDs, but same data.
    if column_location_type:
        df = df[df[column_location_type].isin(location_type_values)]
    cols = [col for col in df.columns if col != column_location_id]
    df = df.drop_duplicates(subset=cols)
    assert df.groupby(column_location_id)[column_location].nunique().max() == 1
    assert df.groupby(column_location)[column_location_id].nunique().max() == 1
    return df

In [324]:
df_population = post_processing(
    df_population,
    "Location",
    "LocID",
    "LocTypeName",
    ["Geographic region", "Income group", "Country/Area", "World"],
)
df_fertility = post_processing(
    df_fertility,
    "Location",
    "LocID",
    "LocTypeName",
    ["Geographic region", "Income group", "Country/Area", "World"],
)
df_demographics = post_processing(
    df_demographics,
    "Location",
    "LocID",
    "LocTypeName",
    ["Geographic region", "Income group", "Country/Area", "World"],
)
df_depratio = post_processing(
    df_depratio,
    "Region, subregion, country or area *",
    "Location code",
    "Type",
    ["Region", "Income Group", "Country/Area", "World"],
)
df_deaths = post_processing(
    df_deaths,
    "Region, subregion, country or area *",
    "Location code",
    "Type",
    ["Region", "Income Group", "Country/Area", "World"],
)

### Uniform column names

In [344]:
columns_rename = {
    "Variant": "Variant",
    "Region, subregion, country or area *": "Location",
    "Notes": "Notes",
    "Location code": "LocID",
    "ISO3 Alpha-code": "ISO3_code",
    "ISO2 Alpha-code": "ISO2_code",
    "SDMX code**": "SDMX_code",
    "Type": "LocTypeName",
    "Parent code": "ParentID",
    "Year": "Time",
}
columns_drop = ["Index"]

In [346]:
df_depratio = df_depratio.rename(columns=columns_rename).drop(columns=columns_drop)
df_deaths = df_deaths.rename(columns=columns_rename).drop(columns=columns_drop)

### Set index

In [350]:
df_population = df_population.set_index(
    ["Location", "Time", "Variant", "AgeGrp"],
    verify_integrity=True,
)
df_fertility = df_fertility.set_index(
    ["Location", "Time", "Variant", "AgeGrp"],
    verify_integrity=True,
)
df_demographics = df_demographics.set_index(
    ["Location", "Time", "Variant"],
    verify_integrity=True,
)
df_depratio = df_depratio.set_index(
    ["Location", "Time", "Variant", "Sex"],
    verify_integrity=True,
)
df_deaths = df_deaths.set_index(
    ["Location", "Time", "Variant", "Sex"],
    verify_integrity=True,
)

### Columns as str

In [438]:
def df_cols_as_str(df):
    df.columns = df.columns.astype(str)
    return df

In [442]:
df_population = df_cols_as_str(df_population)
df_fertility = df_cols_as_str(df_fertility)
df_demographics = df_cols_as_str(df_demographics)
df_depratio = df_cols_as_str(df_depratio)
df_deaths = df_cols_as_str(df_deaths)

### Type fixes

In [ ]:
_type = "category"
df_deaths = df_deaths.assign(Notes=df_deaths.Notes.astype(_type))
df_depratio = df_depratio.assign(Notes=df_depratio.Notes.astype(_type))

## Add tables

In [443]:
from owid.catalog import utils

In [462]:
ds = Dataset.create_empty(dest_dir)
ds.metadata = convert_walden_metadata(walden_ds)
ds.metadata.short_name = "un_wpp"
ds.save()

In [480]:
data = [
    (df_population, "population"),
    (df_fertility, "fertility"),
    (df_demographics, "demographics"),
    (df_deaths, "deaths"),
    (df_depratio, "dependency_ratio"),
]
for elem in data:
    print(elem[1])
    t = Table(elem[0])
    t.metadata.short_name = elem[1]
    ds.add(utils.underscore_table(t))

population
fertility
demographics
deaths
dependency_ratio


## Cleanup

In [ ]:
shutil.rmtree(tmp_dir)